# CT Scanner Simulation: Proof of Functionality (Group 6)

## Introduction

In this report we report the functionality of the Python CT scanner simulation that has been developed over the course of the first 2 weeks of project GG2.

The main capabilities of this simulation are listed below:

1. X-Ray Source Definition for X-Ray Generation (`photons.py` & `fake_source.py`)

2. Modelling of X-Ray Scattering and Attenuation by Different Materials (class `Material`)

3. Simulation of the X-ray Detector Array (`ct_detect.py`)

4. Phantom Definition (`ct_phantom.py`)

5. Production of Uncalibrated Sinograms (`ct_scan.py`)

6. Production of Calibrated Sinograms (`ct_calibrate.py`)

7. Implementation of the Inverse Radon Transform for Reconstruction from Sinograms (`back_project.py` & `ramp_filter.py`)

8. Complete Implementation of the Scanning and Reconstruction Process for an Arbitrary Phantom (`scan_and_reconstruct.py`)

## 1. X-Ray Source Definition for X-Ray Generation

![Figure 1]( source.JPG "Figure 1: Schematic of a typical X-ray source. Reproduced from GG2 Project Handout. Author: Graham Treece")
<center> <i> Figure 1: Schematic of a typical X-ray source. Reproduced from GG2 Project Handout. Author: Graham Treece </i></center>


The modules `photons.py` and `fake_source.py` provide the X-ray source generation capability of our simulation. Sources with different energy distributions can be selected from the predefined sources stored in the `source` class. The different sources vary in filter type, filter width and the potential difference between anode and cathode (see Fig.1) in kVp which limits the maximum photon energy in the resulting distribution. The predefined options are listed below:

1. 100kVp, 1mm Al
2. 100kVp, 2mm Al
3. 100kVp, 3mm Al
4. 100kVp, 4mm Al
5. 80kVp, 1mm Al
6. 80kVp, 2mm Al
7. 80kVp, 3mm Al
8. 80kVp, 4mm Al

The source distributions for these options are plotted in Fig. 2:

![Figure 2]( source_distribution.JPG "Figure 2: Source Distribution for Predefined Options. Reproduced from GG2 Project Handout. Author: Graham Treece")

<center> <i> Figure 2: Source Distribution for Predefined Options. Reproduced from GG2 Project Handout. Author: Graham Treece </i></center>

In `fake_source.py` we also provide the option of generating an arbitrary source distribution by allowing the user to define a filter material, filter width, and maximum photon energy (through anode/cathode potential difference). This module further provides the option of defining an ideal source with an energy distribution centred on the maximum photon energy specified by the user.

## 2. Modelling of X-Ray Scattering and Attenuation by Different Materials

The `Material` class contains as attributes the names of 18 different materials and their corresponding attenuation coefficients for 200 energy levels (0.001 MeV - 0.2 MeV). The materials for which data is listed are:

1. Air              &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  10. Chromium 
2. Adipose           &nbsp;  &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp;  11. Iron
3. Soft Tissue &nbsp;    &nbsp; &nbsp; &nbsp;  12. Carbon
4. Breast Tissue &nbsp;  &nbsp;   13. Nickel
5. Water &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; 14. Magnesium
6. Blood &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; 15. Aluminium
7. Bone &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; 16. Copper 
8. Titanium &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; 17. Co-Cr 
9. Cobalt &nbsp; &nbsp;  &nbsp;   &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;  18. Stainless Steel

Typical attenuation coefficient distributions for 10 materials over the aforementioned energy range are shown in Fig.3:

![Figure 3]( attenuation_coeff_distribution.JPG "Figure 3: Attenuation Coefficient vs. Photon Energy for 10 materials in the Material Class. Reproduced from GG2 Project Handout. Author: Graham Treece")

<center> <i> Figure 3: Attenuation Coefficient vs. Photon Energy for 10 materials in the Material Class. Reproduced from GG2 Project Handout. Author: Graham Treece </i> </center>

## 3. Simulation of the X-ray Detector Array

`ct_detect.py` simulates the X-ray detector array used to measure the total residual energy of source rays after passing through a phantom. It implements the rule defined in *Equation 1*:

$$I_{tot} = \sum_{E} I_{0,E}  e^{\sum_{m} \mu_{m,E} x_m}  \; \; (Equation \; 1) $$ 


## 4. Phantom Definition

`ct_phantom.py` defines a phantom (`type array`) of specified dimensions using different material ellipses (with arbitirary minor/major axes and the capability of rotating the ellipse) of positive and negative "mass" to create different shapes. The function ct_phantom can return one of 7 different types of phantom, which are listed below:

1. Simple circle for looking at calibration issues
2. Point attenuator for looking at resolution
3. Single large hip replacement
4. Bilateral hip replacement
5. Sphere with three satellites
6. Disc and other sphere
7. Pelvic fixation pins

Types 3-7 re-create a metal hip implant whose material for the metallic element you can select by choosing the desired value of the 'metal' parameter, default is Titanium.

An example of a type 3 phantom is shown in Fig.4: 

![Figure 4]( normal_phantom.png "Figure 4: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows material index)")

<center> <i> Figure 4: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows material index) </i> </center>


`atten_coeff_ct_phantom.py` performs a similar task but instead of storing the material index in the phantom array it stores the material attenuation coefficient at a user-defined energy level. This phantom is useful to compare the results of a complete simulation using an ideal source matching the user-defined energy level.

An example of a phantom defined using the material attenuation coefficients for 0.069 MeV can be seen in Fig.5:

![Figure 5]( atten_coeff_phantom.png "Figure 5: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows attenuation coeffiecient at 0.069 MeV)")

<center> <i> Figure 5: Type 3 Phantom - Single Large Titanium Hip Replacement (scale shows attenuation coefficient at 0.069 MeV) </i> </center>

## 5. Production of Uncalibrated Sinograms

`ct_scan.py` performs a complete scan of a previously-defined phantom (of dimensions specified by the `scale` parameter) for a user-defined number of angles. The `ct_scan` function returns a sinogram of residual energies as computed by `ct_detect.py`, an example of which can be seen in Fig. 6:

![Figure 6]( scan_256.png "Figure 6: Residual Energy Sinogram for a Type 3 Phantom (as described in Section 4)")

<center> <i> Figure 6: Residual Energy Sinogram for a Type 3 Phantom (as described in Section 4) </i> </center>

This sinogram of residual energies must be converted into a sinogram of attenuation coefficients to be able to later reconstruct the original phantom. This process is performed by the `ct_calibrate.py` module.

## 6. Production of Calibrated Sinograms

`ct_calibrate.py` implements the approximation in Equation 2 to compute the total attenuation coefficient ($\mu_{tot}$) as seen by the X-ray after passing through a section of a phantom. 

$$\mu_{tot} = \sum_{m} \mu_{m,E} x_m \approx \sum_{m} \mu_{m} x_m = -\log_e{\frac{I_{tot}}{\sum_{E} I_{0,E} } }$$

The function defined therein returns a sinogram of attenuation coefficients, an example of which is shown in Fig. 7:

![Figure 7]( sinogram.png "Figure 7: Attenuation Coefficient Sinogram for a Type 3 Phantom (as described in Section 4)")

<center> <i> Figure 7: Attenuation Coefficient Sinogram for a Type 3 Phantom (as described in Section 4) </i> </center>


## 7. Implementation of the Inverse Radon Transform for Reconstruction from Sinograms


The inverse *Radon transform* is used to convert the sinogram of attenuation coefficients returned by the `ct_calibrate.py` module into a scalar field of attenuation coefficients $\mu(x,y)$ which will describe the phantom as in `atten_coeff_ct_phantom.py`. The analogy for this transform is that we are performing a filtered back-projection (see Fig. 8) of the sinogram over all angles used to define it onto an array of the same dimensions as the original phantom.

![Figure 8]( back_projection.JPG "Figure 8: Pictorial Explanation of Filtered Back-Projection. Left: Forward Projection as performed by ct_scan. Right: Back-projection as performed by back_project.py. Reproduced from GG2 Project Handout. Author: Graham Treece")

<center> <i> Figure 8: Pictorial Explanation of Filtered Back-Projection. Left: Forward Projection as performed by ct_scan. Right: Back-projection as performed by back_project.py. Reproduced from GG2 Project Handout. Author: Graham Treece</i> </center>


The implementation of this transform requires the definition and use of the Ram-Lak filter (`ramp_filter.py`) that is convolved with the sinogram. In the frequency domain, this reduces to a multiplication between the fourier transform of the Ram-Lak filter (see Fig. 9) and the fourier transform of the sinogram. The inverse fourier transform of this multiplication integrated over all the angles (`back_project.py`) used to compute the sinogram results in the reconstruction of the original phantom.

![Figure 9]( RamLak.JPG "Figure 9: Continuous and Discretised Ram-Lak Filter. Reproduced from GG2 Project Handout. Author: Graham Treece ")

<center> <i> Figure 9: Continuous and Discretised Ram-Lak Filter. Reproduced from GG2 Project Handout. Author: Graham Treece </i> </center>

The continuous Ram-Lak filter was approximated by a raised cosine function with different roll-off factors defined by the parameter `alpha`. This was done to ensure the attenuation of higher spatial frequencies than those present in the phantom. The plots of this function for different values of `alpha` (as well as a comparison with the original Ram-Lak filter) are shown in Fig. 10:

![Figure 10]( raised_cosine.JPG "Figure 10: Raised Cosine Functions for different values of alpha. Reproduced from GG2 Project Handout. Author: Graham Treece ")

<center> <i> Figure 10: Raised Cosine Functions for different values of alpha. Reproduced from GG2 Project Handout. Author: Graham Treece </i> </center>

## 8. Complete Implementation of the Scanning and Reconstruction Process for an Arbitrary Phantom

`scan_and_reconstruct.py` is a module that implements the full scanning and reconstruction process for a phantom of the user's choice by sequentially calling the aformentioned modules and functions. The parameters that can be varied are:

1. Source Type 
2. Phantom 
3. Scale of Phantom
4. Number of angles used in the scan

The effects of changing the different parameters on the quality of the reconstruction were studied, as well as any trade-offs regarding complexity of computation. Our tests are described in the following sections:

### 8.1 Varying Angles


The two videos below demonstrate the difference in resolution when varying the number of angles in the scanning process. The first video clearly shows the reduction in streak artefacts as the number of angles increases thus improving the resolution. 
The second video demostrates how the difference between the original and the reconstructed image varies with an increasing number of angles. It can be seen that the difference becomes less visible with more angles that further shows the improvement in the resolution.

In [6]:
from IPython.display import Video
Video("ideal_source_reconstruction_video.mp4") #if video does not play please find it in the github repository

<center> <i> Video Showing the Reconstructions for Different Numbers of Angles in the Scan. </i> </center>

In [5]:
Video("angle_differences_video.mp4") #if video does not play please find it in the github repository

<center> <i> Video Showing the Difference (i.e. subtraction) between the original phantom as defined in atten_coeff_ct_phantom and Reconstructions Using Different Numbers of Angles in the Scan. </i> </center>

### 8.2 Varying Alpha

The following plots show the effect of varying the alpha parameter of the Ram-Lak filter on the reconstruction. Increasing  alpha reduces the gain for high frequency components and results in smoother images as seen below. This helps to reduce the streak artefacts introduced by the backprojection algorithm but must not be set too high as it also results in the loss of resolution.



<table><tr><td><img src='reconstructed_with_ideal_source_256_0.001.png'></td><td><img src='reconstructed_with_ideal_source_256_0.05.png'></td></tr>

<center> <i> Figure 11: Left: 256 Angles, Alpha 0.001. Right: 256 Angles, Alpha 0.05. </i> </center>

<tr><td><img src='reconstructed_with_ideal_source_256_0.1.png'></td><td><img src='reconstructed_with_ideal_source_256_0.5.png'></td></tr>
   

<center> <i> Figure 12: Left: 256 Angles, Alpha 0.1. Right: 256 Angles, Alpha 0.5. </i> </center>

   <center> <tr><td><img src='reconstructed_with_ideal_source_256_2.5.png'></td> </center> </tr>


<center> <i> Figure 13: 256 Angles, Alpha 2.5. </i> </center>

### 8.3 Varying Interpolation Method

We also experimented with varying the interpolation method in the `back_project.py` module. We looked at the default linear interpolation algorithm and compared it to the cubic interpolation algorithm as defined in `scipy.interpolate.interp1d`. As seen from the images below, the cubic interpolation results in sharpened edges and boundaries of the original structure in comparison to the linear interpolation, however, it also slightly sharpens the streaks introduced by the backprojection algorithm.


<table><tr><td><img src='reconstructed_with_ideal_source_256_0.1.png'></td><td><img src='reconstructed_with_ideal_source_256_0.1_cubic.png'></td></tr></table>

<center> <i> Figure 14: Left: Linear Interpolation. Right: Cubic Interpolation </i> </center>

## Some Fun!

This a warning for those of you who have upcoming CT scans. Please seek medical advice urgently if you observe the following reconstruction! 

<table><tr><td><img src='squidward_original.png'></td><td><img src='squidward_reconstruct.png'></td></tr></table>

<center> <i> Figure 15: Left: Original Phantom. Right: Reconstructed Phantom </i> </center>

